优秀的提示词:
【立角色】:引导AI进入具体场景,赋予其行家身份
【述问题】:告诉AI你的困惑和问题,以及背景信息
【定目标】:告诉AI你的需求,希望达成的目标
【补要求】:告诉AI回答时注意什么,或者如何回复

提示词模版:
1.将提示词提炼成模板
2.实现提示词的复用、版本管理、动态变化等

## Prompt Template

In [9]:
#字符模板
from langchain.prompts import PromptTemplate 

prompt = PromptTemplate.from_template("你是一个{name},帮我起{num}个具有{county}特色的{gender}孩名字")
prompt.format(name="大师",county="法国",num=3,gender="男")

'你是一个大师,帮我起3个具有法国特色的男孩名字'

## ChatPromptTemplate

In [18]:
from langchain.prompts import ChatPromptTemplate 

chat_template = ChatPromptTemplate.from_messages(
[
    ("system","你是一个起名大师。你的名字叫{name}."),
    ("human","你好{name},你感觉如何?"),
    ("ai","你好!我状态非常好!"),
    ("human","{user_input}")
])
chat_template.format_messages(name="陈大师",user_input="你叫什么名字呢?")


[SystemMessage(content='你是一个起名大师。你的名字叫陈大师.'),
 HumanMessage(content='你好陈大师,你感觉如何?'),
 AIMessage(content='你好!我状态非常好!'),
 HumanMessage(content='你叫什么名字呢?')]

In [21]:
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

from langchain.schema import SystemMessage
from langchain.schema import HumanMessage
from langchain.schema import AIMessage
#直接创建消息
SystemMessage(
content="你是一个起名大师",
additional_kwargs={"大师姓名":"陈瞎子"}
)
HumanMessage(
content="请问大师叫什么?"
)
AIMessage(
content="我叫陈瞎子"
)

SystemMessage(content='你是一个起名大师', additional_kwargs={'大师姓名': '陈瞎子'})

HumanMessage(content='请问大师叫什么?')

AIMessage(content='我叫陈瞎子')

## ChatMessagePromptTemplate

In [27]:
from langchain.prompts import AIMessagePromptTemplate
from langchain.prompts import SystemMessagePromptTemplate
from langchain.prompts import HumanMessagePromptTemplate
from langchain.prompts import ChatMessagePromptTemplate
prompt="愿{subject}与你同在!"
chat_message_prompt = AIMessagePromptTemplate.from_template(template=prompt)
chat_message_prompt.format(subject="原力")

chat_message_prompt = SystemMessagePromptTemplate.from_template(template=prompt)
chat_message_prompt.format(subject="原力")

chat_message_prompt = HumanMessagePromptTemplate.from_template(template=prompt)
chat_message_prompt.format(subject="原力")

chat_message_prompt = ChatMessagePromptTemplate.from_template(role="天行者",template=prompt)
chat_message_prompt.format(subject="原力")


AIMessage(content='愿原力与你同在!')

SystemMessage(content='愿原力与你同在!')

HumanMessage(content='愿原力与你同在!')

ChatMessage(content='愿原力与你同在!', role='天行者')

## 自定义模版

In [72]:
##函数大师:根据函数名称,查找函数代码,并给出中文的代码说明
from langchain.prompts import StringPromptTemplate
#定义一个简单的函数作为示例效果
def hello_world():
    print("Hello,world!")
    return "Hello, world!"

PROMPT = """\
你是一个非常有经验和天赋的程序员,现在给你如下函数名称你会按照如下格式,输出这段代码的名称、源代码、中文解释
函数名称:{function_name}
源代码:
{source_code}
代码解释:
"""

import inspect

def get_source_code(function_name):
    #获得源代码
    return inspect.getsource(function_name)

class CustmPrompt(StringPromptTemplate):
    def format(self,**kwargs)->str:
        #获得源代码
        source_code = get_source_code(kwargs["function_name"])
        #生成提示词模板
        prompt=PROMPT.format(
        function_name=kwargs["function_name"].__name__,source_code=source_code
        )
        return prompt

a = CustmPrompt(input_variables=["function_name"])
pm = a.format(function_name=hello_world)
print(pm)

from openai import OpenAI
import os
from dotenv import load_dotenv

load_dotenv(override=True)

client = OpenAI(
    api_key=os.getenv("ALI_QW_API_KEY"),  
    base_url="https://dashscope.aliyuncs.com/compatible-mode/v1",
)

completion = client.chat.completions.create(
    model="qwen-long",
    messages=[
        {
            'role': 'user',
            'content': pm
        }
    ],
    stream=False
)

print(completion.choices[0].message.content)



你是一个非常有经验和天赋的程序员,现在给你如下函数名称你会按照如下格式,输出这段代码的名称、源代码、中文解释
函数名称:hello_world
源代码:
def hello_world():
    print("Hello,world!")
    return "Hello, world!"

代码解释:



True

函数名称: hello_world

源代码:
```python
def hello_world():
    print("Hello, world!")
    return "Hello, world!"
```

中文解释: 
此函数`hello_world`用于打印并返回字符串"Hello, world!"。当调用此函数时，它首先在控制台上打印出问候语，然后返回同样的字符串。


## jinjia2 来实现模版格式化

In [2]:
!pip install jinja2

In [ ]:
##Jinja2是一个灵活、高效的Python模板引擎,可以方便地生成各种标记格式的文档。
from langchain.prompts import PromptTemplate

jinja2_template = "给我讲一个关于{{name}}的{{what}}故事"
prompt = PromptTemplate.from_template(jinja2_template, template_format="jinja2")
prompt.format(name="狗剩",what="高兴")

## 组合式提示词模板
Final prompt:最终返回的提示词模板  
Pipeline prompts:组成提示词管道的模板


- 三层提示词设计

In [10]:
#Final Prompt由一系列变量构成
from langchain.prompts import PromptTemplate

full_template= """{Character}
{behavior}
{prohibit}"""

full_prompt = PromptTemplate.from_template(full_template)


In [16]:
from langchain.prompts import load_prompt

prompt = load_prompt("./simple_propmt.yaml")
print(prompt.format(name="小王",what="搞笑的"))


给我讲一个关于小王的搞笑的故事
